# Obituary Generator with Claude

This notebook generates obituaries from uploaded PDF or text files and allows conversational follow-up with Claude.

## Setup Instructions
1. Install required packages
2. Add your Anthropic API key
3. Upload a document about the person
4. Generate and refine the obituary through conversation

## Step 1: Install Required Packages

In [ ]:
!pip install anthropic pypdf ipywidgets

## Step 2: Import Libraries

In [ ]:
import anthropic
import os
from pypdf import PdfReader
from ipywidgets import widgets
from IPython.display import display, Markdown, clear_output
import io

## Step 3: Configure API Key

Enter your Anthropic API key below. You can get one from: https://console.anthropic.com/

In [ ]:
# Enter your API key here
API_KEY = ""  # Replace with your actual API key

# Or use environment variable
if not API_KEY:
    API_KEY = os.environ.get("ANTHROPIC_API_KEY", "")

if not API_KEY:
    print("⚠️ Please set your API key above or in the ANTHROPIC_API_KEY environment variable")
else:
    print("✓ API key configured")
    client = anthropic.Anthropic(api_key=API_KEY)

## Step 4: Helper Functions

In [ ]:
def extract_text_from_pdf(pdf_file):
    """
    Extract text content from a PDF file.
    
    Args:
        pdf_file: File object or path to PDF
    
    Returns:
        Extracted text as string
    """
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def read_text_file(text_file):
    """
    Read content from a text file.
    
    Args:
        text_file: File object or path to text file
    
    Returns:
        File content as string
    """
    if isinstance(text_file, str):
        with open(text_file, 'r', encoding='utf-8') as f:
            return f.read()
    else:
        return text_file.read().decode('utf-8')

class ObituaryGenerator:
    """
    Manages obituary generation and conversational follow-ups.
    Keeps track of the full conversation history with Claude.
    """
    
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)
        self.conversation_history = []  # Stores all messages
        self.document_content = None
        
    def load_document(self, file_content, file_type='text'):
        """
        Load and store document content.
        
        Args:
            file_content: The text content or file object
            file_type: 'text' or 'pdf'
        """
        if file_type == 'pdf':
            self.document_content = extract_text_from_pdf(io.BytesIO(file_content))
        else:
            self.document_content = file_content if isinstance(file_content, str) else file_content.decode('utf-8')
        
        print(f"✓ Document loaded ({len(self.document_content)} characters)")
    
    def generate_initial_obituary(self):
        """
        Generate the first obituary from the document.
        This starts a new conversation.
        """
        if not self.document_content:
            return "Please load a document first!"
        
        # Create initial prompt
        initial_prompt = f"""Based on the following information about a person, please write a thoughtful and respectful obituary. 
Include key life events, achievements, family connections, and what made them special.

Document content:
{self.document_content}

Please generate a well-structured obituary."""
        
        # Start new conversation
        self.conversation_history = [
            {"role": "user", "content": initial_prompt}
        ]
        
        # Call Claude API
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=self.conversation_history
        )
        
        # Add Claude's response to history
        assistant_message = response.content[0].text
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message
    
    def follow_up(self, user_message):
        """
        Send a follow-up message in the conversation.
        This maintains the full conversation context.
        
        Args:
            user_message: The follow-up question or instruction
        
        Returns:
            Claude's response
        """
        if not self.conversation_history:
            return "Please generate an initial obituary first!"
        
        # Add user's follow-up to conversation
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # Call Claude with full conversation history
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=self.conversation_history
        )
        
        # Add Claude's response to history
        assistant_message = response.content[0].text
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message
    
    def reset_conversation(self):
        """Clear the conversation history and start fresh."""
        self.conversation_history = []
        print("✓ Conversation reset")

## Step 5: Initialize the Generator

In [ ]:
# Create the obituary generator instance
generator = ObituaryGenerator(API_KEY)
print("✓ Generator initialized")

## Step 6: Upload and Load Your Document

Choose one of the following options:

### Option A: Upload a file using the file uploader

In [ ]:
# File uploader widget
uploader = widgets.FileUpload(
    accept='.pdf,.txt',
    multiple=False,
    description='Upload File'
)

def on_file_upload(change):
    """Handle file upload event"""
    if uploader.value:
        uploaded_file = list(uploader.value.values())[0]
        file_name = list(uploader.value.keys())[0]
        
        # Determine file type
        file_type = 'pdf' if file_name.endswith('.pdf') else 'text'
        
        # Load the document
        generator.load_document(uploaded_file['content'], file_type)
        print(f"✓ Loaded: {file_name}")

uploader.observe(on_file_upload, names='value')
display(uploader)

### Option B: Paste text directly

In [ ]:
# Or paste your text here
document_text = """
Paste information about the person here...
"""

# Uncomment the line below to load the pasted text
# generator.load_document(document_text, 'text')

## Step 7: Generate Initial Obituary

In [ ]:
# Generate the initial obituary
print("Generating obituary...\n")
obituary = generator.generate_initial_obituary()
display(Markdown("## Generated Obituary\n\n" + obituary))

## Step 8: Interactive Follow-up Conversation

Now you can refine the obituary through conversation! Run this cell and ask follow-up questions.

In [ ]:
# Create interactive widgets for follow-up
follow_up_input = widgets.Textarea(
    value='',
    placeholder='Ask a follow-up question or request changes (e.g., "Make it shorter", "Add more about their career", "Make it more personal")',
    description='Follow-up:',
    layout=widgets.Layout(width='100%', height='80px')
)

submit_button = widgets.Button(
    description='Send',
    button_style='primary',
    tooltip='Send follow-up message'
)

output_area = widgets.Output()

def on_submit_click(b):
    """Handle follow-up submission"""
    user_message = follow_up_input.value.strip()
    
    if not user_message:
        with output_area:
            print("Please enter a message!")
        return
    
    with output_area:
        clear_output(wait=True)
        print(f"You: {user_message}\n")
        print("Claude is thinking...\n")
    
    # Get response from Claude
    response = generator.follow_up(user_message)
    
    with output_area:
        clear_output(wait=True)
        display(Markdown(f"**You:** {user_message}\n\n**Claude:**\n\n{response}"))
    
    # Clear input
    follow_up_input.value = ''

submit_button.on_click(on_submit_click)

# Display the interface
display(widgets.VBox([follow_up_input, submit_button, output_area]))

print("\n💡 Examples of follow-up prompts you can try:")
print("  • Make it shorter and more concise")
print("  • Add more emphasis on their family life")
print("  • Make the tone more celebratory")
print("  • Include specific dates and locations")
print("  • Rewrite in a different style")

## Step 9: View Full Conversation History

In [ ]:
# View the complete conversation
print(f"Total messages in conversation: {len(generator.conversation_history)}\n")
print("="*80)

for i, message in enumerate(generator.conversation_history, 1):
    role = "You" if message["role"] == "user" else "Claude"
    content = message["content"]
    
    # Truncate very long messages for display
    if len(content) > 500:
        content = content[:500] + "... (truncated)"
    
    print(f"\n[Message {i}] {role}:")
    print(content)
    print("="*80)

## Step 10: Reset and Start Over (Optional)

In [ ]:
# Reset the conversation and start fresh
generator.reset_conversation()
print("You can now go back to Step 7 to generate a new obituary!")

## Example Usage Summary

**How this works:**

1. **Conversation Memory**: The `ObituaryGenerator` class keeps a `conversation_history` list that stores ALL messages between you and Claude

2. **Initial Generation**: When you first generate an obituary, it creates a conversation with:
   - Your initial request + document content
   - Claude's obituary response

3. **Follow-ups**: Each time you send a follow-up:
   - Your message is added to the history
   - The ENTIRE history is sent to Claude (so it remembers everything)
   - Claude's new response is added to the history

4. **Why this matters**: Claude can remember and reference:
   - The original document content
   - The obituary it wrote
   - All your previous requests and changes

This is exactly what Assignment 4 is asking for - demonstrating proper conversation management!